In [ ]:
!pip install simpletransformers

In [ ]:
!nvidia-smi

Sun Aug 22 00:42:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
import random
import torch

from sklearn.metrics import accuracy_score, f1_score, classification_report
from simpletransformers.t5 import T5Model, T5Args

import re
import pandas as pd

import numpy as np
import string
from tqdm import tqdm_notebook
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!ls data/

test_1st.xlsx  train_1.xlsx  val_1.xlsx
test_2st.xlsx  train_2.xlsx  val_2.xlsx


In [ ]:
train = pd.read_excel('data/train_2.xlsx')
train.head()

,prefix,input_text,input_text_body,target_text,raw_title,raw_text,raw_target_text
0,summarization,поправки верховенстве конституции основ нравст...,поправки верховенстве конституции основ нравст...,изменение конституции,Поправки о верховенстве Конституции и основ нр...,\nПоправки о верховенстве Конституции и основ ...,Изменение Конституции
1,summarization,уефа оштрафовал модель забег купальнике поле ф...,уефа оштрафовал модель забег купальнике поле ф...,футбольные еврокубки 2018 19,УЕФА оштрафовал модель за забег в купальнике н...,\nУЕФА оштрафовал модель за забег в купальнике...,Футбольные еврокубки-2018/19
2,summarization,тульская область изменяет порядок предоставлен...,центр депутаты тульской областной думы четвер...,налоговые льготы,Тульская область изменяет порядок предоставлен...,Тула. 25 февраля. ИНТЕРФАКС-ЦЕНТР - Депутаты Т...,Налоговые льготы
3,summarization,пресс конференция путина 19 декабря стала самы...,вел онлайн трансляцию пресс конференции прези...,путин пресс конференция 2019,Пресс-конференция Путина 19 декабря стала одно...,\nПресс-конференция Путина 19 декабря стала од...,Путин: пресс-конференция - 2019
4,summarization,глава дипломатии ес намерен визита москву обсу...,европейская внешнеполитическая служба евс рас...,визит борреля в москву,Глава дипломатии ЕС намерен во время визита в ...,Брюссель. 1 февраля. ИНТЕРФАКС - Европейская в...,Визит Борреля в Москву


как видно ниже, собрали все заголовки сюжетов в 1 текст для обучения модели (каждый заголовок пишется до точки)

In [ ]:
train['input_text'].values[:2]

array(['поправки верховенстве конституции основ нравственности прошли первое чтение.крашенинников заверил сириус станет избирательным округом.дума уточнила наказание призывы отчуждению российских территорий.дума приняла законы верховенстве конституции международным правом.дума приняла закон упразднении региональных конституционных судов.думе предложили внести поправку терешковой закон выборах президента.дума поддержала упразднение конституционных судов регионах.новым министрам предстоит утверждение госдуме.думе предложили расширить гарантии неприкосновенности экс президентов.совет федерации принял пакет конституционных законов.думу внесли законопроект праве экс президента сенатором пожизненно.закон конституционном суде принят третьем чтении.госдума приняла конституционный закон правительстве.крашенинников объяснил нужен запрет публикации особых мнений судей кс.кремле сформировали мнение запрете особых мнений судей кс.рабочая группа конституции обсудит законопроекты правительстве кс.кон

In [ ]:
val = pd.read_excel('data/val_2.xlsx')
val.head()

,prefix,input_text,input_text_body,target_text,raw_title,raw_text,raw_target_text
0,summarization,кравчук высказался переговоры россией донбассу...,украина украинская власть должна продолжать в...,саммит нормандской четверки,Кравчук высказался за переговоры с Россией по ...,Киев. 12 марта. ИНТЕРФАКС-УКРАИНА - Украинская...,"Саммит ""Нормандской четверки"""
1,summarization,белоруссии ввести уголовную ответственность ра...,запад белоруссии подготовили новую редакцию з...,ответственность за распространение лжи о белор...,В Белоруссии могут ввести уголовную ответствен...,Минск. 18 марта. ИНТЕРФАКС-ЗАПАД - В Белорусси...,Ответственность за распространение лжи о Белор...
2,summarization,красноярский край оценил ущерб результате авар...,дочка норникеля ао нтэк получила министерства...,аварии на тэц норникеля,Красноярский край оценил ущерб в результате ав...,"Москва. 11 февраля. ИНТЕРФАКС - ""Дочка"" ""Норни...",Аварии на ТЭЦ Норникеля
3,summarization,нобелевскую премию получила всемирная продовол...,нобелевскую премию получила всемирная продовол...,нобелевская премия 2020,Нобелевскую премию мира получила Всемирная про...,\nНобелевскую премию мира получила Всемирная п...,Нобелевская премия - 2020
4,summarization,десятков активистов оппозиции арестованы гонко...,десятков активистов оппозиции арестованы гонко...,протесты в гонконге,Несколько десятков активистов оппозиции аресто...,\nНесколько десятков активистов оппозиции арес...,Протесты в Гонконге


In [ ]:
test = pd.read_excel('data/test_2st.xlsx')
test.head()

,prefix,input_text,input_text_body,target_text,raw_title,raw_text,raw_target_text
0,summarization,скр опроверг завершение психиатрической экспер...,у галявиев доставлен казани москву проведения ...,нападения в школах,СКР опроверг завершение психиатрической экспер...,\nСКР опроверг завершение психиатрической эксп...,Нападения в школах
1,summarization,зедер возложил ответственность сша происходяще...,зедер возложил ответственность сша происходяще...,афганистан во власти талибов,Зедер возложил ответственность на США за проис...,\nЗедер возложил ответственность на США за про...,Афганистан во власти талибов
2,summarization,автобус сбил насмерть остановке долгопрудном.м...,автобус сбил насмерть остановке долгопрудном 7...,аварии в москве,Автобус сбил насмерть двух человек на остановк...,\nАвтобус сбил насмерть двух человек на остано...,Аварии в Москве
3,summarization,польше освобожден российский болельщик задержа...,польше освобожден российский болельщик задержа...,евро 2012,В Польше освобожден последний российский болел...,\nВ Польше освобожден последний российский бол...,Евро-2012
4,summarization,ураган дориан нанес катастрофический ущерб баг...,ураган дориан нанес катастрофический ущерб баг...,ураганы в сша,"Ураган ""Дориан"" нанес катастрофический ущерб Б...","\nУраган ""Дориан"" нанес катастрофический ущерб...",Ураганы в США


In [ ]:
# как таргет используем исходный raw_target_text - необработанные названия сюжетов
train.columns = ['prefix', 'input_text', 'input_text_body', 'raw_target_text', 'raw_title',
       'raw_text', 'target_text']

val.columns = ['prefix', 'input_text', 'input_text_body', 'raw_target_text', 'raw_title',
       'raw_text', 'target_text']

test.columns = ['prefix', 'input_text', 'input_text_body', 'raw_target_text', 'raw_title',
       'raw_text', 'target_text']

In [ ]:
test.head()

,prefix,input_text,input_text_body,raw_target_text,raw_title,raw_text,target_text
0,summarization,скр опроверг завершение психиатрической экспер...,у галявиев доставлен казани москву проведения ...,нападения в школах,СКР опроверг завершение психиатрической экспер...,\nСКР опроверг завершение психиатрической эксп...,Нападения в школах
1,summarization,зедер возложил ответственность сша происходяще...,зедер возложил ответственность сша происходяще...,афганистан во власти талибов,Зедер возложил ответственность на США за проис...,\nЗедер возложил ответственность на США за про...,Афганистан во власти талибов
2,summarization,автобус сбил насмерть остановке долгопрудном.м...,автобус сбил насмерть остановке долгопрудном 7...,аварии в москве,Автобус сбил насмерть двух человек на остановк...,\nАвтобус сбил насмерть двух человек на остано...,Аварии в Москве
3,summarization,польше освобожден российский болельщик задержа...,польше освобожден российский болельщик задержа...,евро 2012,В Польше освобожден последний российский болел...,\nВ Польше освобожден последний российский бол...,Евро-2012
4,summarization,ураган дориан нанес катастрофический ущерб баг...,ураган дориан нанес катастрофический ущерб баг...,ураганы в сша,"Ураган ""Дориан"" нанес катастрофический ущерб Б...","\nУраган ""Дориан"" нанес катастрофический ущерб...",Ураганы в США


In [ ]:
train = train[['prefix','input_text','target_text']]
val = val[['prefix','input_text','target_text']]
test = test[['prefix','input_text','target_text']]

In [ ]:
test.head()

,prefix,input_text,target_text
0,summarization,скр опроверг завершение психиатрической экспер...,Нападения в школах
1,summarization,зедер возложил ответственность сша происходяще...,Афганистан во власти талибов
2,summarization,автобус сбил насмерть остановке долгопрудном.м...,Аварии в Москве
3,summarization,польше освобожден российский болельщик задержа...,Евро-2012
4,summarization,ураган дориан нанес катастрофический ущерб баг...,Ураганы в США


задаём параметры обучения модели и сохранения checkpoints

Кроме fine-tuning mT5, хотели попробовать mBART, но не хватило времени

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 300
model_args.train_batch_size = 2
model_args.eval_batch_size = 2
model_args.num_train_epochs = 7

model_args.evaluate_during_training = True #True
# model_args.evaluate_during_training_steps = 15000
model_args.evaluate_during_training_verbose = True

model_args.output_dir = 'mT5-checkpoints'

model_args.use_multiprocessing = False
model_args.fp16 = False

model_args.save_eval_checkpoints = True
model_args.save_model_every_epoch = False

model_args.no_cache = True
model_args.save_steps = 92

model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_return_sequences = 1

model_args.wandb_project = "summarization"

model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=True)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [ ]:
model.train_model(train, eval_data=val)

  0%|          | 0/183 [00:00<?, ?it/s]

Using Adafactor for T5


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3351: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3351: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Running Epoch 6 of 7:   0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3351: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


(644,
 {'global_step': [92, 184, 276, 368, 460, 552, 644],
  'eval_loss': [2.519651687663534,
   2.9181701152221016,
   3.732302681259487,
   3.1543539995732517,
   3.2450361510981685,
   3.2382556251857593,
   3.491371968518133],
  'train_loss': [4.662533283233643,
   1.6185569763183594,
   4.120190143585205,
   1.1435672044754028,
   1.619796633720398,
   0.29167085886001587,
   0.11365748196840286]})

готовим данные для предсказания модели

In [ ]:
to_predict_train = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(train["prefix"].tolist(), train["input_text"].tolist())
]

to_predict_val = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(val["prefix"].tolist(), val["input_text"].tolist())
]

In [ ]:

to_predict_test = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(test["prefix"].tolist(), test["input_text"].tolist())
]

осуществляем само предсказание

In [ ]:
preds_train = model.predict(to_predict_train)

In [ ]:
preds_val = model.predict(to_predict_val)

Generating outputs:   0%|          | 0/23 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
preds_test = model.predict(to_predict_test)

Generating outputs:   0%|          | 0/23 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/45 [00:00<?, ?it/s]

In [ ]:
train['preds'] = preds_train
train.head(20)

,prefix,input_text,target_text,preds
0,summarization,поправки верховенстве конституции основ нравст...,Изменение Конституции,Изменение Конституции
1,summarization,уефа оштрафовал модель забег купальнике поле ф...,Футбольные еврокубки-2018/19,Футбольные еврокубки-2018/19
2,summarization,тульская область изменяет порядок предоставлен...,Налоговые льготы,Налоговые льготы
3,summarization,пресс конференция путина 19 декабря стала самы...,Путин: пресс-конференция - 2019,Путин: пресс-конференция-2020
4,summarization,глава дипломатии ес намерен визита москву обсу...,Визит Борреля в Москву,Визит Борреля в Москву
5,summarization,компания вектор биальгам месяц полтора намерен...,Производство вакцин от COVID19,Производство вакцины от COVID19
6,summarization,спикер палаты представителей сша объявила созд...,События 6 января в США,Скандал в США
7,summarization,задержанных москве участников форума муниципал...,Задержания на форуме муниципальных депутатов,Задержания на форуме муниципальных депутатов
8,summarization,армения вернула азербайджану военнопленных гла...,МИД Армении о войне в Карабахе,Дипломатическое противостояние
9,summarization,посол сша рф продолжает работу вашингтон прида...,Российско-американские связи,Дипломатическое противостояние


In [ ]:
val['preds'] = preds_val
val.head(20)

,prefix,input_text,target_text,preds
0,summarization,кравчук высказался переговоры россией донбассу...,"Саммит ""Нормандской четверки""",Путин: кравчук: Путин: Путин: Путин: -
1,summarization,белоруссии ввести уголовную ответственность ра...,Ответственность за распространение лжи о Белор...,Белоруссии
2,summarization,красноярский край оценил ущерб результате авар...,Аварии на ТЭЦ Норникеля,Авария на Красноярском края
3,summarization,нобелевскую премию получила всемирная продовол...,Нобелевская премия - 2020,Нобелевская премия - - - премия - - -
4,summarization,десятков активистов оппозиции арестованы гонко...,Протесты в Гонконге,Протесты в гонконге
5,summarization,аргентина получила ещ 330 тыс доз спутника v.м...,"Вакцина ""Спутник V""","Производство вакцины ""Спутник V"" - РФпи"
6,summarization,грузинские оппозиционеры приехали тюрьме руста...,Политический кризис в Грузии,Протесты в Армении
7,summarization,принадлежащий авангарду калинин перешел хк ска...,КХЛ-2016/17,Футбольные еврокубки-2018
8,summarization,палата представителей одобрила резолюцию призы...,Отстранение Трампа,Отношения США
9,summarization,новых заражений covid 19 таджикистане минздрав...,Заражения COVID-19,Блокировка COVID-19


In [ ]:
val.tail(20)

,prefix,input_text,target_text,preds
26,summarization,цска разгромил ахмат матче чемпионата россии ф...,Российская премьер-лига-2018/19,российских футболу
27,summarization,турецкая армия покидать позиции северо востоке...,Вторжение Турции в Сирию,Военные действий в РФ
28,summarization,рост курса биткойна показывает потенциал волат...,Пресс-конференция Набиуллиной,Курсы российских российских российских российс...
29,summarization,трамп раскритиковал пенса нежелание самостояте...,Итоги выборов США,Выборы в США
30,summarization,президент польской млековиты опроверг вымогате...,Обвинения против главы Россельхознадзора,Блокировка взятки
31,summarization,странах балтии выявлено 2 тыс случаев covid 19...,Статистика новых случаев COVID-19 в мире,Блокировка COVID-19
32,summarization,решение прекращении вещания телеканалов 112 ук...,Санкции против телеканалов на Украине,Блокировка Twitter
33,summarization,матчи петербурге посетили 30 иностранных болел...,Евро-2020,Чемпионат мира по футболу
34,summarization,ангелик кербер стала первой ракеткой.вавринка ...,US Open-2016,Футбольные еврокубки-2018
35,summarization,мк 1 число суточных заражений covid 19 молдави...,COVID-19 в Молдавии,Блокировка COVID-19


In [ ]:
test['preds'] = preds_test
test

,prefix,input_text,target_text,preds
0,summarization,скр опроверг завершение психиатрической экспер...,Нападения в школах,Стрельба на казани
1,summarization,зедер возложил ответственность сша происходяще...,Афганистан во власти талибов,США
2,summarization,автобус сбил насмерть остановке долгопрудном.м...,Аварии в Москве,Авария на Московском метро
3,summarization,польше освобожден российский болельщик задержа...,Евро-2012,Футбольные евролигации
4,summarization,ураган дориан нанес катастрофический ущерб баг...,Ураганы в США,"Ураган ""флори в флориде"""
5,summarization,обвиняемый изнасиловании дочери житель ленобла...,Криминал в многодетной семье под Гатчиной,"Агент ЦРУ ""гатчиной"""
6,summarization,республиканская больница владикавказе прекрати...,Пандемия коронавируса,Блокировка вакцины от COVID-19
7,summarization,мэр ричмонда распорядился убрать памятники кон...,Волна беспорядков в США,Протесты в США
8,summarization,полумиллиона вышли барселоне манифестацию.матч...,Протесты в Каталонии,Протесты в Барселле
9,summarization,суд оставил силе пожизненный срок убийцы девоч...,Убийство девятилетней девочки в Саратове,"Убийство ""нормандской игры"" в саратове"


убедимся, что модель генерирует заголовки на тестовой и валидационной выборке отличные от обучаюющей выборки

In [ ]:
test['was_in_train?'] = [1 if elem in train['preds'].unique() else 0 for elem in test['preds']]

In [ ]:
test[test['was_in_train?']  == 0]

,prefix,input_text,target_text,preds,was_in_train?
0,summarization,скр опроверг завершение психиатрической экспер...,Нападения в школах,Стрельба на казани,0
1,summarization,зедер возложил ответственность сша происходяще...,Афганистан во власти талибов,США,0
2,summarization,автобус сбил насмерть остановке долгопрудном.м...,Аварии в Москве,Авария на Московском метро,0
3,summarization,польше освобожден российский болельщик задержа...,Евро-2012,Футбольные евролигации,0
4,summarization,ураган дориан нанес катастрофический ущерб баг...,Ураганы в США,"Ураган ""флори в флориде""",0
5,summarization,обвиняемый изнасиловании дочери житель ленобла...,Криминал в многодетной семье под Гатчиной,"Агент ЦРУ ""гатчиной""",0
6,summarization,республиканская больница владикавказе прекрати...,Пандемия коронавируса,Блокировка вакцины от COVID-19,0
7,summarization,мэр ричмонда распорядился убрать памятники кон...,Волна беспорядков в США,Протесты в США,0
8,summarization,полумиллиона вышли барселоне манифестацию.матч...,Протесты в Каталонии,Протесты в Барселле,0
9,summarization,суд оставил силе пожизненный срок убийцы девоч...,Убийство девятилетней девочки в Саратове,"Убийство ""нормандской игры"" в саратове",0


In [ ]:
val['was_in_train?'] = [1 if elem in train['preds'].unique() else 0 for elem in val['preds']]

In [ ]:
val[val['was_in_train?']  == 0]

,prefix,input_text,target_text,preds,was_in_train?
0,summarization,кравчук высказался переговоры россией донбассу...,"Саммит ""Нормандской четверки""",Путин: кравчук: Путин: Путин: Путин: -,0
1,summarization,белоруссии ввести уголовную ответственность ра...,Ответственность за распространение лжи о Белор...,Белоруссии,0
2,summarization,красноярский край оценил ущерб результате авар...,Аварии на ТЭЦ Норникеля,Авария на Красноярском края,0
3,summarization,нобелевскую премию получила всемирная продовол...,Нобелевская премия - 2020,Нобелевская премия - - - премия - - -,0
4,summarization,десятков активистов оппозиции арестованы гонко...,Протесты в Гонконге,Протесты в гонконге,0
5,summarization,аргентина получила ещ 330 тыс доз спутника v.м...,"Вакцина ""Спутник V""","Производство вакцины ""Спутник V"" - РФпи",0
6,summarization,грузинские оппозиционеры приехали тюрьме руста...,Политический кризис в Грузии,Протесты в Армении,0
7,summarization,принадлежащий авангарду калинин перешел хк ска...,КХЛ-2016/17,Футбольные еврокубки-2018,0
8,summarization,палата представителей одобрила резолюцию призы...,Отстранение Трампа,Отношения США,0
10,summarization,газовики завершают работы восстановлению газос...,Аварии на магистралях Газпрома,Аварийная на ТЭТП в оренбуржье,0


сохраним полученные результаты

In [ ]:
train.to_excel('results/train_ann1.xlsx', index=False)
val.to_excel('results/val_ann1.xlsx', index=False)
test.to_excel('results/test_ann1.xlsx', index=False)
